# Forecasting

In [1]:
import warnings
warnings.filterwarnings('ignore')
import pickle

#Preparing, cleaning, analyzing
import pandas as pd
import numpy as np


## Data Preparation

In [2]:
df = pd.read_csv('./data/Alko_Insg.csv', index_col='date')

In [3]:
df.head()

,year,month,value
date,,,
2000-01-01,2000,1,78
2000-02-01,2000,2,53
2000-03-01,2000,3,73
2000-04-01,2000,4,78
2000-05-01,2000,5,96


In [10]:
#Model loading
from tensorflow.keras.models import load_model
model = load_model('best_model.h5')

In [11]:
#Pickle scaler for features and value loading

# Open the file in binary mode
with open('./scalers/f_transformer.pkl', 'rb') as file: 
    # Call load method to deserialze
    f_transformer = pickle.load(file)

# Open the file in binary mode
with open('./scalers/cnt_transformer.pkl', 'rb') as file: 
    # Call load method to deserialze
    cnt_transformer = pickle.load(file)    
    

## Inference

df

In [12]:
def creat_ds(x,y, time_step=1):
    #Create x series and y series to hold sequences
    xs, ys = [], []
    
    for i in range(len(x) - time_step):
        #Extract the sequence
        v = x.iloc[i: (i+time_step)].to_numpy()
        #append it into x series
        xs.append(v)
        
        #Repeat all above for y series
        ys.append(y.iloc[i+time_step])
    return np.array(xs), np.array(ys)   


In [16]:
Time_Step = 12
import datetime


def predict():
    input_item= (2021,1)
    f_columns = ['year', 'month']

    input_df = pd.DataFrame()
    (year, month) = input_item
    input_date = datetime.datetime(year, month, 1)
    df.index = pd.to_datetime(df.index)
    input_df = df[df.index < input_date].tail(12)
    input_df.loc[len(input_df.index)] = [year, month, 0]
    input_df.rename(index={12:input_date},inplace=True)

    #Scaling features
    input_df.loc[:, f_columns] = f_transformer.transform(input_df[f_columns].to_numpy())
    input_df['value'] = cnt_transformer.transform(input_df[['value']])

        
    xs_inf, ys_inf = creat_ds(input_df, input_df.value, time_step=Time_Step)
    y_predict_inf = model.predict(xs_inf)
    y_pred_inv = cnt_transformer.inverse_transform(y_predict_inf)
    print('Pridicted Value ', np.ceil(y_pred_inv[0][0]))
    if df[df.index==input_date].shape[0] !=0:
        rea_value = df[df.index==input_date]
        print('Real value ',rea_value.value[0])

In [17]:
df

,year,month,value
date,,,
2000-01-01,2000,1,78
2000-02-01,2000,2,53
2000-03-01,2000,3,73
2000-04-01,2000,4,78
2000-05-01,2000,5,96
...,...,...,...
2020-08-01,2020,8,46
2020-09-01,2020,9,46
2020-10-01,2020,10,34


In [18]:
predict()

Pridicted Value  15.0
